# Plotting a time series using Bokeh
Let's say we want to plot the number of hit during our data taking with the Timepix detector. This isn't quite as trivial as it sounds, unfortunately. For example, while each frame of data records a discrete number of pixels, this is only for the duration of the **acquisition time** of the frame. There are slight gaps between the frames due to the readout time. It is therefore more appropriate to plot the **pixels per second** associated with each frame as a rectangle on a graph of _time_ versus _pixels per second_; the area of each rectangle then gives you the number of pixels hit in that frame.

This makes the high-level Bokeh `Bar` chart a bit useless for what we want. We therefore will need to play with our data a bit before using the `quad` plotting functionality of Bokeh to get the plot that we want.

## Getting the data
For convenience, we've converted the [background radiation measurement data](https://dx.doi.org/10.6084/m9.figshare.1618851.v2) featured in [this paper](http://dx.doi.org/10.1080/00107514.2015.1045193) into a CSV file that can be read straight in as a **pandas** `DataFrame` like so:

In [6]:
import pandas as pd

df = pd.read_csv('data/bkgradiation/B06-W0212_2014-04-02-140255.csv')

df.head()

,start_time_s,acq_time_s,n_pixels,pixels_per_second
0,1396447375,60.0,57,0.950000
1,1396447435,60.0,41,0.683333
2,1396447495,60.0,18,0.300000
3,1396447555,60.0,21,0.350000
4,1396447615,60.0,21,0.350000


## Preparing the data
The **Bokeh** `quad` functionality allows us to plot rectangles on our axes using four lists representing the top, bottom, left and right corners of each rectangle. Using some _list comprehension_ on the columns extracted from our `DataFrame`, we can get these values as follows:

* Left values: these are the start times of each frame. However, we only have the UNIX timestamp for each frame, i.e. the number of seconds since 1st January 1970. We can retrieve the number of seconds since the start of the day using the modulus operator on the extracted `start_time_s` column values;
* Right values: we can then add the **acquisition time** to each start time using list comprehension and the `zip` function;
* Top values: these are simply the _pixels per second_ as extracted from the data;
* Bottom values: always 0 - we want the rectangle to touch the x axis, so we create a list of zeroes (one zero for every frame).

In [17]:
# Get some constant values for convenience - see timevals.py.
from timevals import *

## The left-hand values for each frame.
l_values = [(s % SECONDS_IN_A_DAY) for s in df['start_time_s'].tolist()]

## The right-hand values for each frame.
r_values = [s + a for s, a in zip(l_values, df['acq_time_s'].tolist())]

## The top values for each frame - pixels per second (from the data).
t_values = df['pixels_per_second'].tolist()

## The bottom values for each frame - a list of zeroes.
b_values = [0.0] * len(l_values)

## Plotting the data
Now let's use Bokeh to plot the data. First, let's ask Bokeh to output to the notebook:

In [13]:
# Standard imports 

from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

Bokeh's `datetime` axis functionality can't quite do what we want, so the `get_day_plot_figure` function uses a few tricks to format the _x_ axis tick labels to show the time. Look at `plottools.py` to see how it's done - all we need to supply is the upper _y_ axis limit (extracted from the maximum pixels per second value). We can then make the plot using the `quad` function:

In [18]:
from plottools import get_day_plot_figure

## The plot representing the whole day (24 hours).
p = get_day_plot_figure(max(t_values))

# Plot the 
p.quad(top=t_values, bottom=b_values, left=l_values, right=r_values, color="#B3DE69")

show(p)

As you can see, the plot covers the whole day - but the detector was only taking data for an hour from about 2pm. What's nice about Bokeh is that you can use the zoom (both **Box** and **Wheel**) and **Pan** tools in the toolbar on the right to inspect the details of the plot more closely. However, it's the potential for interacting with the plot that makes this approach particularly exciting. Watch this space!